# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行


In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "========================================="


== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter��核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。


In [2]:
# 2. 设置pip 为清华源
%pip config list -v set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list -v


Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
ERROR: Need an action (debug, edit, get, list, set, unset) to perform.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT


env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 3.78 GB (Available: 1.86 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

# 第二节：明确和直接

## 设置

运行以下设置单元格来加载你的API密钥并建立`get_completion`辅助函数。

In [5]:
# 安装OpenAI库
%pip install openai==1.107.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取大模型的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: 大模型的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://api.apiyi.com/v1
  🤖 模型: deepseek-chat
  🌐 API地址: https://api.apiyi.com/v1
  🔑 API密钥: sk-AGnnT...A14f

✅ 使用统一配置管理成功！


---

## 课程 

**大模型对清晰直接的指令响应最佳。**

把大模型想象成任何一个刚入职的人。除了你直接告诉它的内容外，**大模型对该做什么没有任何上下文**。就像你第一次指导人类完成任务一样，你越是以直接的方式向大模型准确解释你想要什么，大模型的响应就会越好、越准确。"				
				
当有疑问时，遵循**清晰提示的黄金法则**：
- 将你的提示展示给同事或朋友，让他们自己按照指令操作，看看他们能否产生你想要的结果。如果他们感到困惑，大模型也会困惑。				

### 示例

让我们通过几个对比鲜明的例子来演示明确和直接的重要性。

#### 示例1：模糊 vs 明确的任务指令

In [6]:
# 模糊的提示
PROMPT = "帮我处理一下这个数据"

# 显示大模型回答
print("模糊提示的回答:")
print(get_completion(PROMPT))
print()

模糊提示的回答:
当然可以！为了更好地帮助你处理数据，请提供以下信息：

1. **数据内容**：你的数据是文本、表格、JSON、CSV，还是其他格式？可以粘贴示例或描述结构。
2. **处理需求**：你需要完成什么任务？例如：
   - 数据清洗（去重、填充缺失值、格式标准化等）
   - 统计分析（平均值、分组汇总等）
   - 可视化（图表类型？）
   - 其他操作（合并表格、提取特定字段等）
3. **工具偏好**：希望用 Python（Pandas）、Excel、SQL，还是其他工具处理？

### 示例参考
如果你提供这样的数据：
```csv
日期,销售额,产品
2023-01-01,1200,手机
2023-01-01,800,耳机
2023-01-02,1500,手机
```

并说明需求：
> “帮我按产品分类计算总销售额，并导出为Excel。”

我可以直接给出Python代码或Excel操作步骤。请补充你的具体数据和需求！



可以看到，大模型无法理解"处理数据"具体指什么，只能给出通用的回答。

现在让我们提供一个**明确和直接**的指令：

In [7]:
# 明确和直接的提示
PROMPT = """分析以下销售数据，计算每个月的同比增长率：
1月：100万
2月：120万
3月：110万
4月：130万

要求：
- 以表格形式输出结果
- 包含月份、销售额、同比增长率三列
- 同比增长率保留2位小数，用百分比表示"""

# 显示大模型回答
print("明确提示的回答:")
print(get_completion(PROMPT))

明确提示的回答:
由于缺少去年同期的销售额数据，无法直接计算同比增长率。同比增长率的计算公式为：

\[ \text{同比增长率} = \left( \frac{\text{本期销售额} - \text{去年同期销售额}}{\text{去年同期销售额}} \right) \times 100\% \]

假设我们补充去年同期的销售额数据如下（虚构数据用于示例）：

| 月份 | 今年销售额（万） | 去年销售额（万） |
|------|------------------|------------------|
| 1月  | 100              | 90               |
| 2月  | 120              | 100              |
| 3月  | 110              | 95               |
| 4月  | 130              | 110              |

根据上述数据，计算同比增长率后的表格如下：

| 月份 | 销售额（万） | 同比增长率  |
|------|--------------|-------------|
| 1月  | 100          | 11.11%      |
| 2月  | 120          | 20.00%      |
| 3月  | 110          | 15.79%      |
| 4月  | 130          | 18.18%      |

### 计算过程：
1. **1月**：  
   \(\frac{100 - 90}{90} \times 100\% = 11.11\%\)

2. **2月**：  
   \(\frac{120 - 100}{100} \times 100\% = 20.00\%\)

3. **3月**：  
   \(\frac{110 - 95}{95} \times 100\% = 15.79\%\)

4. **4月**：  
   \(\frac{130 - 110}{110} \times 100\% = 18.18\%\)

### 注意事项：
- 若需实际计算，请提供去年同期的销售额数据。


看到差别了吗？明确的指令让大模型能够准确理解任务，并按照要求的格式输出结果。

#### 示例2：输出格式的明确指定

让我们看另一个例子，关于如何明确指定输出格式。首先是一个**模糊的请求**：

In [8]:
# 模糊的提示 - 没有指定输出格式
PROMPT = "给我一些Python学习建议"

# 显示大模型回答
print("模糊提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")

模糊提示的回答:
学习Python是一个循序渐进的过程，以下是一些系统化的建议，适合不同阶段的学习者：

### 一、基础阶段（1-3周）
1. **掌握核心语法**
   - 数据类型（整数、字符串、列表、字典等）
   - 流程控制（if/else/for/while）
   - 函数定义与参数传递
   - 文件读写操作
   - 推荐资源：Python官方文档的[Tutorial](https://docs.python.org/3/tutorial/)

2. **实践小项目**
   ```python
   # 示例：简易计算器
   def calculator():
       while True:
           try:
               num1 = float(input("输入第一个数字: "))
               op = input("输入运算符(+-*/): ")
               num2 = float(input("输入第二个数字: "))
               if op == '+': print(num1 + num2)
               elif op == '-': print(num1 - num2)
               elif op == '*': print(num1 * num2)
               elif op == '/': print(num1 / num2)
               else: print("无效运算符")
           except ValueError:
               print("请输入有效数字！")

   calculator()
   ```

### 二、进阶提升（2-4周）
1. **面向对象编程**
   - 类与对象的概念
   - 继承与多态
   - 魔术方法（如`__str__`, `__init__`）

2. **常用库实战**
   - `requests`网络请求
   - `pandas`数据处理
   - `matplotlib`可视化
   ```python
   # 示例：用requests获取API数据
   impor

大模型给出了一般性的建议，但格式不够结构化。现在让我们用**明确的格式要求**：

In [9]:
# 明确和直接的提示 - 指定了具体的输出格式
PROMPT = """为初学者制定一个Python学习计划。

要求：
1. 分为3个阶段：初级、中级、高级
2. 每个阶段包含：学习时长、核心概念、推荐资源、实战项目
3. 用表格形式呈现
4. 不要添加额外解释，直接给出表格

输出格式示例：
| 阶段 | 学习时长 | 核心概念 | 推荐资源 | 实战项目 |
|------|----------|----------|----------|----------|
| 初级 | ... | ... | ... | ... |"""

# 显示大模型回答
print("明确提示的回答:")
print(get_completion(PROMPT))

明确提示的回答:
| 阶段 | 学习时长 | 核心概念 | 推荐资源 | 实战项目 |  
|------|----------|----------|----------|----------|  
| 初级 | 4-6周 | 变量、数据类型、运算符、条件语句、循环、函数、列表、字典、字符串操作 | 《Python Crash Course》、Codecademy Python课程 | 计算器、待办事项列表、简单文本游戏 |  
| 中级 | 6-8周 | 面向对象编程、文件操作、异常处理、模块和包、列表推导式、基础算法 | 《Python编程：从入门到实践》、Real Python教程 | 天气查询应用、博客系统、数据分析脚本 |  
| 高级 | 8-12周 | 多线程/多进程、网络编程（HTTP/API）、数据库操作、测试框架、高级库（NumPy/Pandas）、Web框架（Django/Flask） | 《流畅的Python》、官方文档、Coursera Python专项课程 | 爬虫项目、RESTful API服务、机器学习模型部署 |


#### 示例3：避免歧义的明确指令

最后一个例子展示如何通过明确指令避免歧义。先看一个**容易产生歧义**的提示：


In [10]:
# 容易产生歧义的提示
PROMPT = "解释一下机器学习"

# 显示大模型回答
print("歧义提示的回答:")
print(get_completion(PROMPT))
print("\n" + "="*50 + "\n")


歧义提示的回答:
机器学习（Machine Learning, ML）是人工智能（AI）的一个核心分支，其核心目标是让计算机系统能够**从数据中自动学习规律和模式**，并基于这些学习结果做出预测或决策，而无需显式编程。以下是机器学习的核心概念和分类的详细解释：

---

### **1. 核心思想**
- **数据驱动**：与传统编程不同，机器学习通过分析大量数据自动发现隐藏的规律（如“输入X如何对应输出Y”），而非依赖人工编写的规则。
- **泛化能力**：好的模型不仅能拟合已知数据（训练集），还能对未知数据（测试集）做出准确预测。

---

### **2. 主要类型**
#### **(1) 监督学习（Supervised Learning）**
- **任务**：通过带标签的数据（输入X和对应的输出Y）学习映射关系。
- **典型应用**：
  - **分类**（预测离散标签）：垃圾邮件识别、图像分类（如CNN）。
  - **回归**（预测连续值）：房价预测、股票趋势分析。
- **常见算法**：线性回归、决策树、支持向量机（SVM）、神经网络。

#### **(2) 无监督学习（Unsupervised Learning）**
- **任务**：从无标签数据中发现隐藏结构或模式。
- **典型应用**：
  - **聚类**：客户分群（如K-means）、社交网络分析。
  - **降维**：数据可视化（如PCA）、特征压缩。
  - **异常检测**：信用卡欺诈识别。
- **常见算法**：K-means、DBSCAN、自编码器（Autoencoder）。

#### **(3) 强化学习（Reinforcement Learning, RL）**
- **任务**：智能体通过与环境交互（试错）学习最优策略，以最大化奖励信号。
- **典型应用**：游戏AI（如AlphaGo）、自动驾驶、机器人控制。
- **核心要素**：状态（State）、动作（Action）、奖励（Reward）、策略（Policy）。

#### **(4) 其他类型**
- **半监督学习**：结合少量标注数据和大量未标注数据。
- **自监督学习**：从数据本身生成标签（如BERT的掩码语言模型）。

---

### **3. 关键步骤**
1. **数据准备**：

这个回答可能过于宽泛或技术性。现在让我们提供一个**明确的上下文和要求**：


In [11]:
# 明确的上下文和要求
PROMPT = """你是一名面向大学一年级计算机专业学生的讲师。用简单易懂的语言解释机器学习。

要求：
1. 用日常生活中的例子来说明概念
2. 避免复杂的数学公式
3. 重点说明机器学习的3个主要类型及其应用场景
4. 控制在200字以内
5. 用1-2-3的序号格式组织内容"""

# 显示大模型回答
print("明确提示的回答:")
print(get_completion(PROMPT))


明确提示的回答:
1. **什么是机器学习？**  
   就像教小孩认水果：你反复给他看苹果和橙子的图片，告诉他名称，最终他能自己区分。机器学习就是让计算机通过大量数据"学习"规律，无需显式编程。

2. **三大类型及应用**  
   - **监督学习**（带答案的学习）：用带标签的数据训练，如垃圾邮件分类（已知哪些是垃圾邮件）。  
   - **无监督学习**（自己找规律）：分析无标签数据，比如电商根据用户购买习惯自动分组。  
   - **强化学习**（试错学习）：像训练小狗，做对给奖励。用于游戏AI（如AlphaGo）和自动驾驶。

3. **核心思想**  
   计算机通过经验改进表现，就像人类学习骑自行车——摔几次后就会掌握平衡。关键是为它提供合适的数据和反馈机制。


### 关键要点总结

通过以上三个对比鲜明的例子，我们可以看到明确和直接的提示带来的巨大差异：

**模糊提示的问题：**
- 任务不明确：「帮我处理数据」vs「分析销售数据并计算同比增长率」
- 格式不清晰：「给建议」vs「用表格形式输出学习计划」  
- 缺乏上下文：「解释机器学习」vs「为大一学生用简单语言解释」

**明确提示的要素：**
1. **具体的任务描述** - 明确说明要做什么
2. **清晰的输出格式** - 指定表格、列表、字数等要求
3. **明确的上下文** - 说明目标受众和使用场景
4. **具体的约束条件** - 字数限制、格式要求等

记住：**大模型无法读懂你的想法，只能按照你给出的指令执行**。越明确越直接的指令，越能得到符合期望的结果。
